In [1]:
import os
import sys
import time
import warnings
import pathlib
import random
import pickle
from tqdm import tqdm

# science
import numpy as np
import pandas as pd
import scipy as sp

# for neural data
import mne
import nilearn
from scipy import signal, stats

# for reading this dataset (paper with neural nets classification movement vs no-movement)
#from scipy.io import netcdf
#import xarray as xr

# for plotting
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns

from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot

# prosto dunul
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset

import pytorch_lightning as pl

#!pip install torchsummary
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
#from torchinfo import summary

sns.set_context('notebook', font_scale=1.2)
sns.set_style('ticks')
sns.set_palette('muted')

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Load data (1 patient)

In [2]:
data_path = r"D:\data_nma2022\tfr data"

# fname_mdata = "power-roi-all-patients-metadata.csv"
# fpath_mdata = os.path.join(data_path, fname_mdata)

print("1. Getting data ...")
labels = []
features = []
days = []
for i_session in [3, 4, 5, 6, 7]:
    fname_data = f"subj_04_day_{i_session}_l_epo-tfr.h5"
    fpath_data = os.path.join(data_path, fname_data)
    print(f"Reading {fname_data}...")
    mne_data = mne.time_frequency.read_tfrs(fpath_data)[0]
    
    data = mne_data.data
    metadata = mne_data.metadata

    print(f"Data (day {i_session}) shape: {data.shape} = (n_samples, n_chan, n_freqs, n_times)")
    
    labels.append(metadata['reach_a'].to_numpy())
    features.append(data)
    days.append(metadata['day'].to_numpy())
    
    
labels, features, days = np.concatenate(labels), np.concatenate(features), np.concatenate(days)

print("Labels, shape = ", labels.shape)
print("Features, shape = ", features.shape)
# print("2. Getting metadata ...")
# print(f"Reading {fname_mdata}...")
# metadata = pd.read_csv(fpath_mdata, index_col=0)
# metadata.head()

1. Getting data ...
Reading subj_04_day_3_l_epo-tfr.h5...
Reading D:\data_nma2022\tfr data\subj_04_day_3_l_epo-tfr.h5 ...
Adding metadata with 21 columns
Data (day 3) shape: (19, 84, 25, 91) = (n_samples, n_chan, n_freqs, n_times)
Reading subj_04_day_4_l_epo-tfr.h5...
Reading D:\data_nma2022\tfr data\subj_04_day_4_l_epo-tfr.h5 ...
Adding metadata with 21 columns
Data (day 4) shape: (42, 84, 25, 91) = (n_samples, n_chan, n_freqs, n_times)
Reading subj_04_day_5_l_epo-tfr.h5...
Reading D:\data_nma2022\tfr data\subj_04_day_5_l_epo-tfr.h5 ...
Adding metadata with 21 columns
Data (day 5) shape: (32, 84, 25, 91) = (n_samples, n_chan, n_freqs, n_times)
Reading subj_04_day_6_l_epo-tfr.h5...
Reading D:\data_nma2022\tfr data\subj_04_day_6_l_epo-tfr.h5 ...
Adding metadata with 21 columns
Data (day 6) shape: (60, 84, 25, 91) = (n_samples, n_chan, n_freqs, n_times)
Reading subj_04_day_7_l_epo-tfr.h5...
Reading D:\data_nma2022\tfr data\subj_04_day_7_l_epo-tfr.h5 ...
Adding metadata with 21 columns
Da

## Train-test split

### Within-subject test-split (leave one day out - LODO)

In [3]:
# days = metadata.day.values
# labels = metadata.reach_a.values

test_day = 4

mask_train = (days != test_day)
mask_test  = (days == test_day)

X_train, y_train = features[mask_train], np.array(labels[mask_train])
X_test,  y_test  = features[mask_test],  np.array(labels[mask_test])

print(f" X_train: {X_train.shape} \n X_test:  {X_test.shape} \n y_train: {y_train.shape} \n y_test:  {y_test.shape} ")

# creating torch tensors (batch size is variable, so we will create data loader in run_nn func)

X_train_lodo = torch.tensor(X_train).float()
X_test_lodo = torch.tensor(X_test).float()

y_train_lodo = torch.tensor(y_train).float()
y_test_lodo = torch.tensor(y_test).float()

n_channels, n_freqs, n_times = X_train[0].shape

 X_train: (226, 84, 25, 91) 
 X_test:  (42, 84, 25, 91) 
 y_train: (226,) 
 y_test:  (42,) 


## Model building blocks

In [4]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


In [8]:
class EcogUNet(pl.LightningModule):
    
    def __init__(self, n_channels, n_freqs, n_times, args, target_type='sin-cos', bilinear=False):
        """
        UNet-based regression model. 
        input: 
            torch.tensor, shape = (batch_size, n_chan, n_freqs, n_times)
            target = 'angle' or 'sin'
        Depending on the target type the model architecture and loss-function are re-adjusted
        
        """
        
        """1 - Initialize parameters"""
        super(EcogUNet, self).__init__()
        self.n_channels = n_channels
        self.n_freqs = n_freqs
        self.n_times = n_times
        self.bilinear = bilinear
        self.target_type = target_type
        self.args = args
        
        self.writer = SummaryWriter()
        
        """2 - Model layers"""

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        
        factor = 2 if bilinear else 1

        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)  
        # - at this point we have vector of shape (batch_size, 64, n_freqs, n_times)
        # - reduce the shape of input by // 2 and channels by 2
        self.down5 = Down(64, 32)   
        self.flatten = nn.Flatten()
        
        # get shape (batch_size, 32 * n_freqs // 2 * n_times // 2)
        # FC1
        self.fc1 = nn.Linear(32 * (n_freqs // 2) * (n_times // 2), 512)
        self.act1 = nn.SiLU()
        self.dropout = nn.Dropout(p=0.3)
        # FC2
        self.fc2 = nn.Linear(512, 256)
        self.act2 = nn.SiLU()
        #self.batchnorm = nn.BatchNorm1d(128)
       
        
        """3 - NN-head and loss function"""
        
        if target_type == 'angle':
      
            self.output = nn.Linear(256, 1)
            self.out_act = nn.Hardtanh(-np.pi, np.pi)
            
            self.loss_fn = lambda prediction, label: 1 - torch.cos(prediction, label)
            
        if target_type == 'sin-cos':
    
            self.output = nn.Linear(256, 2)
            self.out_act = nn.Hardtanh(-1., 1.)
            
            self.loss_fn = nn.MSELoss(reduction='mean')
            
        else:
            raise ValueError("No such target_type. Use 'angle' or 'sin-cos'.")
            

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)

        x = self.up2(x4, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        # returned to original shape and 64 channels
        x = self.down5(x)
        x = self.flatten(x)
        # FC 1 
        x = self.fc1(x)
        x = self.act1(x)
        x = self.dropout(x)
        # FC 2
        x = self.fc2(x)
        x = self.act2(x)
        #x = self.batchnorm(x)
        # output layer [-1, 1]
        x = self.output(x)
        # get angle in rad
        out = self.out_act(x)
        return out


    def training_step(self, batch, batch_idx):
        x, y = batch
        
        # convert angle in degrees to desired lable
        y = y / 180 * np.pi 
        if self.target_type == 'sin-cos':
            sin = torch.sin(y).unsqueeze(1)
            cos = torch.cos(y).unsqueeze(1)
            label = torch.cat((sin, cos), dim=1)
        if self.target_type == 'angle':
            label = y
            
        # make prediction
        prediction = self.forward(x)

        loss = self.loss_fn(prediction, label.view(prediction.shape)).mean()
        
        # regularization
        if self.args['lambda_l1'] != 0:
            l1_loss = sum([torch.sum(torch.abs(p)) for p in model.parameters()])
            loss = loss + args['lambda_l1'] * l1_loss
        if self.args['lambda_l2'] != 0:
            l2_loss = sum([torch.sum(p.pow(2)) for p in model.parameters()])
            loss = loss + args['lambda_l2'] * l2_loss
            
        self.log('train_loss', loss)
        self.writer.add_scalar('Loss/train', loss.item(), batch_idx)
        return loss
    

    def validation_step(self, batch, batch_idx):
        x, y = batch
        # convert angle in degrees to desired lable
        y = y / 180 * np.pi 
        if self.target_type == 'sin-cos':
            sin = torch.sin(y).unsqueeze(1)
            cos = torch.cos(y).unsqueeze(1)
            label = torch.cat((sin, cos), dim=1)
        if self.target_type == 'angle':
            label = y
            
        # make prediction
        prediction = self.forward(x)

        loss = self.loss_fn(prediction, label.view(prediction.shape)).mean()
        self.log('validation_loss', loss)
        self.writer.add_scalar('Loss/validation', loss.item(), batch_idx)
        return loss
    

    def test_step(self, batch, batch_idx):
        x, y = batch

        # convert angle in degrees to desired lable
        y = y / 180 * np.pi 
        if self.target_type == 'sin-cos':
            sin = torch.sin(y).unsqueeze(1)
            cos = torch.cos(y).unsqueeze(1)
            label = torch.cat((sin, cos), dim=1)
        if self.target_type == 'angle':
            label = y
            
        # make prediction
        prediction = self.forward(x)

        loss = self.loss_fn(prediction, label.view(prediction.shape)).mean()
        
        output = dict({
            'test_loss': loss.item()
        })
        self.writer.add_scalar('Loss/test', loss.item(), batch_idx)
        return output
    

    def configure_optimizers(self):
        optimizer = optim.RMSprop(self.parameters(), lr=self.args['lr'], weight_decay=self.args['weight_decay'], momentum=self.args['momentum'])
        return optimizer

In [11]:
%load_ext tensorboard

In [9]:
#@title trian
args = {'n_epochs': 100,
        'batch_size': 1,
        'device': DEVICE, 
        'lr': 1.5e-3, 
        'momentum': 0.9, 
        'weight_decay': 1e-9, 
        'lambda_l1': 1e-7, 
        'lambda_l2': 0}

model = EcogUNet(n_channels, n_freqs, n_times, args, target_type='sin-cos').double().to(args['device'])

trainer = pl.Trainer(max_epochs=args['n_epochs'], accelerator='gpu')

train_loader = DataLoader(list(zip(X_train, y_train)), 
                                   batch_size=args['batch_size'], 
                                   shuffle=True)

test_loader  = DataLoader(list(zip(X_test, y_test)), 
                                   batch_size=args['batch_size'],
                                   shuffle=True)

# Perform training
trainer.fit(model, train_loader, test_loader)

# Perform evaluation
trainer.test(model, test_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name    | Type       | Params
----------------------------------------
0  | inc     | DoubleConv | 85.5 K
1  | down1   | Down       | 221 K 
2  | down2   | Down       | 885 K 
3  | down3   | Down       | 3.5 M 
4  | up2     | Up         | 2.3 M 
5  | up3     | Up         | 574 K 
6  | up4     | Up         | 143 K 
7  | down5   | Down       | 27.8 K
8  | flatten | Flatten    | 0     
9  | fc1     | Linear     | 8.8 M 
10 | act1    | SiLU       | 0     
11 | dropout | Dropout    | 0     
12 | fc2     | Linear     | 131 K 
13 | act2    | SiLU       | 0     
14 | output  | Linear     | 514   
15 | out_act | Hardtanh   | 0     
16 | loss_fn | MSELoss    | 0     
----------------------------------------
16.8 M    Trainable params
0         Non-trainable params
16.8 M    Total params
67.017    Total est

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

C:\Users\aleks\anaconda3\envs\compneuro\lib\site-packages\pytorch_lightning\trainer\trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
C:\Users\aleks\anaconda3\envs\compneuro\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:495: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
C:\Users\aleks\anaconda3\envs\compneuro\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:240: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_

Testing: 0it [00:00, ?it/s]

[{}]

In [12]:
%tensorboard --logdir runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "C:\Users\aleks\anaconda3\envs\compneuro\Scripts\tensorboard-script.py", line 5, in <module>
    from tensorboard.main import run_main
  File "C:\Users\aleks\anaconda3\envs\compneuro\lib\site-packages\tensorboard\main.py", line 27, in <module>
    from tensorboard import default
  File "C:\Users\aleks\anaconda3\envs\compneuro\lib\site-packages\tensorboard\default.py", line 33, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "C:\Users\aleks\anaconda3\envs\compneuro\lib\site-packages\tensorboard\plugins\audio\audio_plugin.py", line 23, in <module>
    from tensorboard import plugin_util
  File "C:\Users\aleks\anaconda3\envs\compneuro\lib\site-packages\tensorboard\plugin_util.py", line 78, in <module>
    _MARKDOWN_STORE = _MarkdownStore()
  File "C:\Users\aleks\anaconda3\envs\compneuro\lib\site-packages\tensorboard\plugin_util.py", line 70, in __ini